<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/CNN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import scipy.io.wavfile
import numpy as np
import sys
import glob 
import librosa
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
from IPython.display import Audio
### Time Distributed ConvNet imports ###
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from IPython.display import Image
from glob import glob
import pickle
import itertools
from sklearn.model_selection import train_test_split
from PIL import Image

### Audioimport ###
import IPython

### Warning ###
import warnings
warnings.filterwarnings('ignore')

#Extract signals and labels

In [57]:
#Extract filename and label
data_path = '/content/drive/MyDrive/Nhận dạng/Data Voice'
emotions=os.listdir(data_path)
filenames = []
labels = []
for emotion in emotions:
   file_names = os.listdir(os.path.join(data_path,emotion))
   for name in file_names:
     if name!='transcript': 
         filenames.append(name.split(".")[0])
         labels.append(np.float(name[0]))

filenames = np.array(filenames)
labels = np.array(labels)

#Build audio vector

In [58]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [59]:
#Extract signal
sample_rate = 16000     
max_pad_len = 49100
signals = []

for idx,name in enumerate(filenames):
    emotion_path = os.path.join(data_path,emotions[int(labels[idx])])
    file_path = os.path.join(emotion_path,name+'.wav')
    y,sr = librosa.load(file_path, sr=sample_rate)
    y = zscore(y)
    if len(y) < max_pad_len:    
        y_padded = np.zeros(max_pad_len)
        y_padded[:len(y)] = y
        y = y_padded
    elif len(y) > max_pad_len:
        y = np.asarray(y[:max_pad_len])
    signals.append(y)

signals = np.array(signals)

In [ ]:
#Show Audio

random_idx = np.random.randint(len(labels))
random_idx=1
random_label = labels[random_idx]
random_signal = signals[random_idx]
random_filename = filenames[random_idx]

# Plot signal wave
plt.figure(figsize=(10,5))
plt.plot(np.arange(len(random_signal))/float(sample_rate), random_signal)
plt.xlim((np.arange(len(random_signal))/float(sample_rate))[0], (np.arange(len(random_signal))/float(sample_rate))[-1])
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Amplitude (dB)', fontsize=16)
plt.title("Signal wave of file '{}' with label {}".format(random_filename, random_label), fontsize=18)
plt.show()

# Play audio file
print("Audio file '{}':".format(random_filename))
Audio(random_signal, rate=sample_rate)

#Augment Noisy

In [3]:
nb_augmented = 2

# Function to add noise to a signals with a desired Signal Noise ratio (SNR)
def noisy_signal(signal, snr_low=15, snr_high=30, nb_augmented=2):
    
    # Signal length
    signal_len = len(signal)

    # Generate White noise
    noise = np.random.normal(size=(nb_augmented, signal_len))
    
    # Compute signal and noise power
    s_power = np.sum((signal / (2.0 ** 15)) ** 2) / signal_len
    n_power = np.sum((noise / (2.0 ** 15)) ** 2, axis=1) / signal_len
    
    # Random SNR: Uniform [15, 30]
    snr = np.random.randint(snr_low, snr_high)
    
    # Compute K coeff for each noise
    K = np.sqrt((s_power / n_power) * 10 ** (- snr / 10))
    K = np.ones((signal_len, nb_augmented)) * K
    
    # Generate noisy signal
    return signal + K.T * noise

In [ ]:
print("Data Augmentation: START")
augmented_signals = list(map(noisy_signal, signals))
print("Data Augmentation: END!")


Data Augmentation: START
Data Augmentation: END!


In [ ]:
#Show raw and noisy audio
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(np.arange(len(random_signal))/float(sample_rate), random_signal)
plt.xlim((np.arange(len(random_signal))/float(sample_rate))[0], (np.arange(len(random_signal))/float(sample_rate))[-1])
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Amplitude (dB)', fontsize=16)
plt.title("Signal wave of file '{}' ".format(random_filename), fontsize=18)

# Plot signal wave with noise
plt.subplot(1,2,2)
plt.plot(np.arange(len(random_signal))/float(sample_rate), augmented_signals[random_idx][0])
plt.xlim((np.arange(len(random_signal))/float(sample_rate))[0], (np.arange(len(random_signal))/float(sample_rate))[-1])
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Amplitude (dB)', fontsize=16)
plt.title("Signal wave of file '{}' with Noise".format(random_filename), fontsize=18)
plt.show()

# Play audio file
print("Audio file '{}':".format(random_filename))
IPython.display.display(Audio(random_signal, rate=sample_rate))

# Play same audio file with noise
print("Audio file '{}' with noise:".format(random_filename))
IPython.display.display(Audio(augmented_signals[random_idx][0], rate=sample_rate))

###Extract mel_spectrogram

In [4]:
def mel_spectrogram(y, sr=16000, n_fft=512, win_length=256, hop_length=128, window='hamming', n_mels=128, fmax=4000):
    
    # Compute spectogram
    mel_spect = np.abs(librosa.stft(y, n_fft=n_fft, window=window, win_length=win_length, hop_length=hop_length)) ** 2
    
    # Compute mel spectrogram
    mel_spect = librosa.feature.melspectrogram(S=mel_spect, sr=sr, n_mels=n_mels, fmax=fmax)
    
    # Compute log-mel spectrogram
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max)
    
    return mel_spect

In [ ]:
# Start feature extraction
print("Feature extraction: START")

# Compute spectogram for all audio file
mel_spects = np.asarray(list(map(mel_spectrogram, signals)))
augmented_mel_spects = [np.asarray(list(map(mel_spectrogram, augmented_signals[i]))) for i in range(len(augmented_signals))]

# Stop feature extraction
print("Feature extraction: END!")

Feature extraction: START
Feature extraction: END!


In [ ]:
# Plot one random Spectogram 
plt.figure(figsize=(20, 10))
plt.imshow(mel_spects[np.random.randint(len(mel_spects))], origin='lower', aspect='auto', cmap='viridis')
plt.title('Log-Mel Spectrogram of an audio file', fontsize=26)
plt.tight_layout()
plt.show()

Split train and test set

In [ ]:
MEL_SPECT_train, MEL_SPECT_test, AUG_MEL_SPECT_train, AUG_MEL_SPECT_test, label_train, label_test = train_test_split(mel_spects, augmented_mel_spects, labels, test_size=0.2,random_state=42)

# Build augmented labels and train
aug_label_train = np.asarray(list(itertools.chain.from_iterable([[label] * nb_augmented for label in label_train])))
AUG_MEL_SPECT_train = np.asarray(list(itertools.chain.from_iterable(AUG_MEL_SPECT_train)))

# Concatenate original and augmented
X_train = np.concatenate((MEL_SPECT_train, AUG_MEL_SPECT_train))
y_train = np.concatenate((label_train, aug_label_train))

# Build test set
X_test = MEL_SPECT_test
y_test = label_test



In [ ]:
from numpy import save
#train and test set of Vie-Vie model
save('vie_X_train.npy',X_train)
save('vie_X_test.npy',X_test)
save('vie_y_train.npy',y_train)
save('vie_y_test.npy',y_test)

Split to frame

In [69]:
# Time distributed parameters
win_ts = 128
hop_ts = 64

# Split spectrogram into frames
def frame(x, win_step=128, win_size=64):
    nb_frames = 1 + int((x.shape[2] - win_size) / win_step)
    frames = np.zeros((x.shape[0], nb_frames, x.shape[1], win_size)).astype(np.float32)
    for t in range(nb_frames):
        frames[:,t,:,:] = np.copy(x[:,:,(t * win_step):(t * win_step + win_size)]).astype(np.float32)
    return frames


In [ ]:
# Frame for TimeDistributed model
X_train = frame(X_train, hop_ts, win_ts)
X_test = frame(X_test, hop_ts, win_ts)

In [ ]:
cd /content/drive/MyDrive/Nhận dạng


/content/drive/MyDrive/Nhận dạng


In [ ]:
#save x_train,y_train,x_test,y_test

save('X_train.npy',X_train)
save('X_test.npy',X_test)
save('y_train.npy',y_train)
save('y_test.npy',y_test)

In [ ]:
from numpy import load,save
X_train = load('X_train.npy',allow_pickle=True)
X_test = load('X_test.npy',allow_pickle=True)
y_train = load('y_train.npy',allow_pickle = True)
y_test = load('y_test.npy',allow_pickle=True)

In [ ]:
#Encode label
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(np.ravel(y_train)))
y_test = np_utils.to_categorical(lb.transform(np.ravel(y_test)))

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] , X_train.shape[2], X_train.shape[3], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] , X_test.shape[2], X_test.shape[3], 1)

Define model

In [ ]:
from IPython.display import Image

K.clear_session()

# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=X_train.shape[1:], name='Input_MELSPECT')

## First LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_1_MELSPECT')(input_y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_1_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_1_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), name='MaxPool_1_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_1_MELSPECT')(y)     

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_2_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_2_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_2_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_2_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_2_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_3_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_3_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_3_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_3_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_3_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_4_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_4_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_4_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_4_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_4_MELSPECT')(y)  

## Flat
y = TimeDistributed(Flatten(), name='Flat_MELSPECT')(y)                      
                               
# Apply 2 LSTM layer and one FC
y = LSTM(256, return_sequences=False, dropout=0.2, name='LSTM_1')(y)
y = Dense(y_train.shape[1], activation='softmax', name='FC')(y)

# Build final model
model = Model(inputs=input_y, outputs=y)

# Plot model graph
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
Image(retina=True, filename='model.png')

Compile

In [ ]:
# Compile model
model.compile(optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.8), loss='categorical_crossentropy', metrics=['accuracy'])

# Save best model
best_model_save = ModelCheckpoint('/content/drive/MyDrive/Model/[CNN-LSTM]Model.hdf5', save_best_only=True, monitor='val_acc', mode='max')

# Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = model.fit(X_train, y_train, batch_size=64, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stopping, best_model_save])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Train model Anh-Việt

###Dowload dataset from kaggle

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/uwrfkaggler/ravdess-emotional-speech-audio")

###Extract signals and labels

In [ ]:
sample_rate = 16000     
max_pad_len = 49100
eng_signals = []
eng_labels = []
dataset_path = '/content/drive/MyDrive/Nhận dạng/ravdess-emotional-speech-audio'
actors = os.listdir(dataset_path)

for actor_file in actors:
    actor_path = os.path.join(dataset_path,actor_file)
    filenames = os.listdir(actor_path)
    for name in filenames:
        temp = int(name[6:8])
        if (temp==1) or (temp==3) or (temp==4) or (temp==5):
            if temp==1:  eng_labels.append(3)
            elif temp==3: eng_labels.append(1)
            elif temp==4: eng_labels.append(2)
            else: eng_labels.append(0)
            file_path = os.path.join(actor_path,name)
            y,sr = librosa.load(file_path, sr=sample_rate)
            y = zscore(y)
            if len(y) < max_pad_len:    
                y_padded = np.zeros(max_pad_len)
                y_padded[:len(y)] = y
                y = y_padded
            elif len(y) > max_pad_len:
                y = np.asarray(y[:max_pad_len])
            eng_signals.append(y)

eng_labels = np.array(eng_labels)
eng_signals = np.array(eng_signals)

###Agumented Noisy

In [ ]:
print("Data Augmentation: START")
eng_augmented_signals = list(map(noisy_signal, eng_signals))
print("Data Augmentation: END!")


Data Augmentation: START
Data Augmentation: END!


###Extract Mel

In [ ]:
# Start feature extraction
print("Feature extraction: START")

# Compute spectogram for all audio file
eng_mel_spects = np.asarray(list(map(mel_spectrogram, eng_signals)))
eng_augmented_mel_spects = [np.asarray(list(map(mel_spectrogram, eng_augmented_signals[i]))) for i in range(len(eng_augmented_signals))]

# Stop feature extraction
print("Feature extraction: END!")

Feature extraction: START
Feature extraction: END!


###Prepare data

In [ ]:
# Build augmented labels and train
eng_aug_labels = np.asarray(list(itertools.chain.from_iterable([[label] * nb_augmented for label in eng_labels])))
eng_AUG_MEL_SPECTs = np.asarray(list(itertools.chain.from_iterable(eng_augmented_mel_spects)))
eng_X_train = np.concatenate((eng_mel_spects,eng_AUG_MEL_SPECTs))
eng_y_train = np.concatenate((eng_labels,eng_aug_labels))


In [ ]:
cd /content/drive/MyDrive/Nhận dạng

/content/drive/MyDrive/Nhận dạng


In [ ]:
#Load test set - Vietnamese
from numpy import load
X_train = load('vie_X_train.npy',allow_pickle=True)
X_test = load('vie_X_test.npy',allow_pickle=True)
y_train = load('vie_y_train.npy',allow_pickle = True)
y_test = load('vie_y_test.npy',allow_pickle=True)
vie_X_test = np.concatenate((X_train,X_test))
vie_y_test = np.concatenate((y_train,y_test))

In [ ]:
# Frame for TimeDistributed model
eng_X_train = frame(eng_X_train, hop_ts, win_ts)
vie_X_test = frame(vie_X_test, hop_ts, win_ts)

In [ ]:
#Encode label

lb = LabelEncoder()
eng_y_train = np_utils.to_categorical(lb.fit_transform(np.ravel(eng_y_train)))
vie_y_test = np_utils.to_categorical(lb.transform(np.ravel(vie_y_test)))

eng_X_train = eng_X_train.reshape(eng_X_train.shape[0], eng_X_train.shape[1] , eng_X_train.shape[2], eng_X_train.shape[3], 1)
vie_X_test = vie_X_test.reshape(vie_X_test.shape[0], vie_X_test.shape[1] , vie_X_test.shape[2], vie_X_test.shape[3], 1)

In [ ]:
from numpy import save

save('eng-vie_X_train.npy',eng_X_train)
save('eng-vie_X_test.npy',vie_X_test)
save('eng_vie_y_train.npy',eng_y_train)
save('eng_vie_y_test.npy',vie_y_test)

In [ ]:
from IPython.display import Image

K.clear_session()

# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=eng_X_train.shape[1:], name='Input_MELSPECT')

## First LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_1_MELSPECT')(input_y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_1_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_1_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), name='MaxPool_1_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_1_MELSPECT')(y)     

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_2_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_2_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_2_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_2_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_2_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_3_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_3_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_3_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_3_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_3_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_4_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_4_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_4_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_4_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_4_MELSPECT')(y)  

## Flat
y = TimeDistributed(Flatten(), name='Flat_MELSPECT')(y)                      
                               
# Apply 2 LSTM layer and one FC
y = LSTM(256, return_sequences=False, dropout=0.2, name='LSTM_1')(y)
y = Dense(eng_y_train.shape[1], activation='softmax', name='FC')(y)

# Build final model
model = Model(inputs=input_y, outputs=y)

# Plot model graph
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
Image(retina=True, filename='model.png')

In [ ]:
# Compile model
model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.8), loss='categorical_crossentropy', metrics=['accuracy'])

# Save best model
best_model_save = ModelCheckpoint('/content/drive/MyDrive/Model/Nhận dạng/Model.h5', save_best_only=True, monitor='val_acc', mode='max')

# Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = model.fit(eng_X_train,eng_y_train, batch_size=64, epochs=200, validation_data=(vie_X_test, vie_y_test), callbacks=[early_stopping, best_model_save])

In [ ]:
model.save('my_model.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Combine Eng + Vie

In [7]:
from numpy import load

eng_X_train = load('eng-vie_X_train.npy')
vie_X_test = load('eng-vie_X_test.npy',)
eng_y_train = load('eng_vie_y_train.npy',)
vie_y_test = load('eng_vie_y_test.npy',)

combined_X = np.concatenate((eng_X_train,vie_X_test))
combined_y = np.concatenate((eng_y_train,vie_y_test))
X_train,X_test,y_train,y_test = train_test_split(combined_X,combined_y,test_size=0.2,random_state=42)

In [ ]:
from IPython.display import Image

K.clear_session()

# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=eng_X_train.shape[1:], name='Input_MELSPECT')

## First LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_1_MELSPECT')(input_y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_1_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_1_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), name='MaxPool_1_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_1_MELSPECT')(y)     

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_2_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_2_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_2_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_2_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_2_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_3_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_3_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_3_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_3_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_3_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_4_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_4_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_4_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_4_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_4_MELSPECT')(y)  

## Flat
y = TimeDistributed(Flatten(), name='Flat_MELSPECT')(y)                      
                               
# Apply 2 LSTM layer and one FC
y = LSTM(256, return_sequences=False, dropout=0.2, name='LSTM_1')(y)
y = Dense(eng_y_train.shape[1], activation='softmax', name='FC')(y)

# Build final model
model = Model(inputs=input_y, outputs=y)

# Plot model graph
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
Image(retina=True, filename='model.png')

In [ ]:
# Compile model
model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.8), loss='categorical_crossentropy', metrics=['accuracy'])

# Save best model
best_model_save = ModelCheckpoint('/content/drive/MyDrive/Model/Nhận dạng/Model.h5', save_best_only=True, monitor='val_acc', mode='max')

# Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = model.fit(X_train,y_train, batch_size=64, epochs=150, validation_data=(X_test, y_test), callbacks=[early_stopping, best_model_save])

In [ ]:
model.save('combined_my_model.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Load model

In [ ]:
combined_model = tf.keras.models.load_model('combined_my_model.h5')
combined_model.evaluate(X_test,y_test,batch_size=32)

y_pred = combined_model.predict(X_test,batch_size=32)
true_labels = np.argwhere(y_test == 1)[:,1]
pre_labels  = np.argwhere(y_pred==np.amax(y_pred,1, keepdims=True))[:,1]

from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

y_target =    [1, 1, 1, 0, 0, 2, 0, 3]
y_predicted = [1, 0, 1, 0, 0, 2, 1, 3]

cm = confusion_matrix(y_target=true_labels, 
                      y_predicted=pre_labels, 
                      binary=False)

fig, ax = plot_confusion_matrix(cm)
plt.show()

#Predict function

In [64]:
def predict_emotion_from_file(model, filename, chunk_step=16000, chunk_size=49100, predict_proba=False, sample_rate=16000):
        # Read audio file
        emotion1={0:'Angry',1:'Happy',2:'Sad',3:'Neutral'}
        y, sr = librosa.core.load(filename, sr=sample_rate, offset=0.5)
        win_ts = 128
        hop_ts = 64
        if len(y) < max_pad_len:    
          y_padded = np.zeros(max_pad_len)
          y_padded[:len(y)] = y
          y = y_padded
        elif len(y) > max_pad_len:
          y = np.asarray(y[:max_pad_len])
        # Split audio signals into chunks
        chunks = frame(y.reshape(1, 1, -1), chunk_step, chunk_size)
        # Reshape chunks
        chunks = chunks.reshape(chunks.shape[1],chunks.shape[-1])

        # Z-normalization
        y = np.asarray(list(map(zscore, chunks)))

        # Compute mel spectrogram
        mel_spect = np.asarray(list(map(mel_spectrogram, y)))

        # Time distributed Framing
        mel_spect_ts = frame(mel_spect,hop_ts,win_ts)
        print(mel_spect_ts.shape)
        # Build X for time distributed CNN
        X = mel_spect_ts.reshape(mel_spect_ts.shape[0],
                                    mel_spect_ts.shape[1],
                                    mel_spect_ts.shape[2],
                                    mel_spect_ts.shape[3],
                                    1)
        # Predict emotion
        if predict_proba is True:
            predict = model.predict(X)
        else:
            predict = np.argmax(model.predict(X), axis=1)
            predict = [emotion1.get(emotion) for emotion in predict]

        return predict